<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2886ab896e294648789ed633913ea2c938aa79356a802b779ad560ccb2933121
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 09:19:27
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -28.33 K (-0.2%)
Current PnL: -27.01 L (-17.45%)
CY Booked + Current PnL: -12.72 L (-8.22%)
-------------------
Total profit:  1.36 L
Total loss:  -28.36 L
-------------------
Total Booked + Current PnL: 14.26 L (11.16%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.64 L (65.44%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.02,-18.10,22.21,0.09,18331.0,-18240.0,82537.0,91.68,35.0,M-SC,2.47,253.0,-1.00,0.59,8.43,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.43,8.61,10.21,19.69,21262.0,16503.0,208245.0,-2.04,56.0,X-MC,2.56,78.0,0.78,1.49,15.99,XY25,NTT,AC
50,MASFIN,398.61,0.57,-2.24,24.84,22.05,23794.0,-2190.0,95790.0,-15.54,54.0,H-SC,6.94,164.0,-0.09,0.69,39.24,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.51,-13.84,16.09,0.02,24199.0,-24159.0,150400.0,129.35,48.0,M-SC,14.42,234.0,-1.00,1.08,30.82,OX40N,NTT,PAINTS
43,ITC,452.00,0.15,-1.40,12.35,10.78,29264.0,-3363.0,236955.0,-41.15,47.0,X-LC,1.97,5.0,-0.11,1.70,3.76,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,2.10,-2.54,112.10,106.72,134754.0,-3127.0,120209.0,-51.77,42.0,H-SC,3.25,139.0,-0.02,0.86,20.25,AR,ATH,MISC
13,BSOFT,831.70,0.79,-22.14,93.67,50.79,102578.0,-31143.0,109510.0,-0.27,64.0,H-SC,6.42,151.0,-0.30,0.79,27.51,XR,ATH,IT
75,TMPV,600.00,0.62,-28.76,72.61,22.98,116858.0,-64959.0,160939.0,-25.52,33.0,X-LC,6.73,3.0,-0.56,1.15,1.22,XY24,NTT,AUTO
50,MASFIN,398.61,0.57,-2.24,24.84,22.05,23794.0,-2190.0,95790.0,-15.54,54.0,H-SC,6.94,164.0,-0.09,0.69,39.24,XR,ATH,FINANCE
60,REPCOHOME,880.00,0.56,-14.76,122.70,89.82,299637.0,-42302.0,244203.0,-57.53,42.0,H-SC,5.87,160.0,-0.14,1.75,29.20,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,-1.68,-15.95,74.84,46.95,78815.0,-19986.0,105311.0,-22.21,42.0,H-SC,7.97,154.0,-0.25,0.76,31.93,MH,ATH,POWER
18,COFFEEDAY,80.00,-1.54,-41.70,132.02,35.27,87400.0,-47347.0,66202.0,-55.73,45.0,L-SC,10.64,270.0,-0.54,0.47,62.03,XR,NTT,HOTELS
61,ROUTE,2227.21,-1.29,-47.89,224.41,69.05,155610.0,-63724.0,69342.0,-57.74,49.0,H-SC,22.67,140.0,-0.41,0.50,6.94,SR,ATH,IT
71,TANLA,1943.92,-1.20,-36.46,245.74,119.69,422791.0,-98713.0,172048.0,-35.93,44.0,H-SC,7.93,156.0,-0.23,1.23,35.92,AR,ATH,IT
44,JCHAC,2282.00,-1.09,-34.65,53.01,-0.01,44273.0,-44287.0,83518.0,16986.96,37.0,M-SC,8.04,233.0,-1.00,0.60,6.90,OX40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.38,-0.88,109.76,107.91,160629.0,-1298.0,146346.0,-19.34,61.0,M-SC,12.48,216.0,-0.01,1.05,9.12,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.38,-0.88,109.76,107.91,160629.0,-1298.0,146346.0,-19.34,61.0,M-SC,12.48,216.0,-0.01,1.05,9.12,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.51,-13.84,16.09,0.02,24199.0,-24159.0,150400.0,129.35,48.0,M-SC,14.42,234.0,-1.00,1.08,30.82,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.02,-18.10,22.21,0.09,18331.0,-18240.0,82537.0,91.68,35.0,M-SC,2.47,253.0,-1.00,0.59,8.43,OX40N,NTT,DURABLES
66,SIS,528.00,0.11,-24.10,59.76,21.26,50365.0,-26754.0,84278.0,1988.08,50.0,H-SC,5.05,166.0,-0.53,0.60,13.87,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.46,-25.41,52.14,13.47,104870.0,-68535.0,201132.0,-69.28,34.0,H-SC,5.65,158.0,-0.65,1.44,2.27,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.0,6.32,62.02,72.27,111734.0,10710.0,180158.0,-8.8,63.0,M-LC,2.59,99.0,0.1,1.29,12.89,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.00,6.32,62.02,72.27,111734.0,10710.0,180158.0,-8.80,63.0,M-LC,2.59,99.0,0.10,1.29,12.89,XR,NTT,IT
25,FINCABLES,1641.55,0.38,-0.88,109.76,107.91,160629.0,-1298.0,146346.0,-19.34,61.0,M-SC,12.48,216.0,-0.01,1.05,9.12,OX40N,ATH,CABLES
38,INDIAMART,4810.62,2.10,-2.54,112.10,106.72,134754.0,-3127.0,120209.0,-51.77,42.0,H-SC,3.25,139.0,-0.02,0.86,20.25,AR,ATH,MISC
1,ABB,7934.00,-0.12,-3.94,51.54,45.57,129522.0,-10315.0,251304.0,-38.27,56.0,H-MC,4.55,120.0,-0.08,1.80,6.93,AR,NTT,ELECTRICAL
85,ZYDUSLIFE,1286.17,0.36,-3.24,40.17,35.63,81461.0,-6784.0,202790.0,-16.74,38.0,H-MC,2.85,119.0,-0.08,1.45,12.90,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.51,-18.67,91.15,55.47,180364.0,-45410.0,197876.0,-27.82,24.0,X-MC,13.41,64.0,-0.25,1.42,1.39,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.12,-9.86,42.12,28.11,76469.0,-19860.0,181550.0,-34.62,27.0,X-MC,13.18,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
8,AWL,485.00,-0.33,-23.32,100.17,53.50,231789.0,-70355.0,231396.0,-62.86,30.0,X-MC,3.26,58.0,-0.30,1.66,2.78,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.41,-39.04,121.80,35.20,95534.0,-50235.0,78435.0,2.82,31.0,X-SC,16.27,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.34,-5.15,30.81,24.08,37100.0,-6534.0,120416.0,-12.87,31.0,X-SC,5.17,89.0,-0.18,0.86,13.46,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.25,-1.48,26.35,24.48,65381.0,-3729.0,248127.0,-81.32,43.0,X-SC,0.27,86.0,-0.06,1.78,21.32,X40N,NTT,MISC
43,ITC,452.0,0.15,-1.40,12.35,10.78,29264.0,-3363.0,236955.0,-41.15,47.0,X-LC,1.97,5.0,-0.11,1.70,3.76,X40,NTT,FMCG
20,DABUR,735.0,-0.14,-2.93,48.04,43.70,115681.0,-7266.0,240802.0,-10.89,37.0,X-MC,2.28,72.0,-0.06,1.73,11.68,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,0.00,-11.26,28.09,13.67,64079.0,-28943.0,228120.0,-17.95,32.0,X-LC,2.30,14.0,-0.45,1.64,7.69,XY25,NTT,FMCG
12,BERGEPAINT,680.0,-0.59,-4.36,26.66,21.14,57965.0,-9915.0,217424.0,-11.18,38.0,X-MC,2.38,74.0,-0.17,1.56,22.55,XY24,NTT,PAINTS


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.13,-25.65,120.79,64.15,213690.0,-61041.0,176910.0,-56.31,33.0,X-SC,2.78,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.41,-39.04,121.80,35.20,95534.0,-50235.0,78435.0,2.82,31.0,X-SC,16.27,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.07,-0.12,-9.86,42.12,28.11,76469.0,-19860.0,181550.0,-34.62,27.0,X-MC,13.18,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
59,RELAXO,1176.00,0.07,-48.03,193.05,52.31,145645.0,-69716.0,75444.0,-44.63,40.0,X-SC,7.09,91.0,-0.48,0.54,0.99,X40N,NTT,FOOTWEAR
31,HAVELLS,2069.16,-0.46,-10.01,47.22,32.48,139372.0,-32836.0,295155.0,-14.62,40.0,X-MC,6.08,67.0,-0.24,2.12,1.12,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.73,-29.90,103.76,42.84,47286.0,-19434.0,45572.0,-54.27,42.0,X-SC,35.36,83.0,-0.41,0.33,3.10,XY24,NTT,MISC
59,RELAXO,1176.00,0.07,-48.03,193.05,52.31,145645.0,-69716.0,75444.0,-44.63,40.0,X-SC,7.09,91.0,-0.48,0.54,0.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.41,-39.04,121.80,35.20,95534.0,-50235.0,78435.0,2.82,31.0,X-SC,16.27,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.34,-5.15,30.81,24.08,37100.0,-6534.0,120416.0,-12.87,31.0,X-SC,5.17,89.0,-0.18,0.86,13.46,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.88,-17.21,73.12,43.32,98595.0,-28032.0,134840.0,-28.80,34.0,X-SC,4.89,90.0,-0.28,0.97,3.53,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.50,-11.43,36.29,20.71,121569.0,-43243.0,334994.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.40,11.93,X40,ATH,IT
41,INFY,1972.00,-0.86,8.44,23.80,34.24,80751.0,26400.0,339288.0,-15.65,59.0,X-LC,4.01,2.0,0.33,2.43,17.42,X40,NTT,IT
75,TMPV,600.00,0.62,-28.76,72.61,22.98,116858.0,-64959.0,160939.0,-25.52,33.0,X-LC,6.73,3.0,-0.56,1.15,1.22,XY24,NTT,AUTO
81,VBL,671.64,-0.54,-4.14,41.53,35.67,125737.0,-13079.0,302763.0,-15.61,53.0,X-LC,3.99,4.0,-0.10,2.17,9.14,X40N,ATH,FMCG
43,ITC,452.00,0.15,-1.40,12.35,10.78,29264.0,-3363.0,236955.0,-41.15,47.0,X-LC,1.97,5.0,-0.11,1.70,3.76,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.10,-37.67,113.21,32.89,54479.0,-29084.0,48122.0,-694.37,53.0,L-MC,6.13,259.0,-0.53,0.35,32.52,XR,NTT,BANKS
6,ASIANTILES,137.00,0.22,-16.63,117.91,81.67,92217.0,-15600.0,78210.0,7027.78,46.0,L-SC,12.04,271.0,-0.17,0.56,51.46,XR,NTT,CERAMICS
50,MASFIN,398.61,0.57,-2.24,24.84,22.05,23794.0,-2190.0,95790.0,-15.54,54.0,H-SC,6.94,164.0,-0.09,0.69,39.24,XR,ATH,FINANCE
13,BSOFT,831.70,0.79,-22.14,93.67,50.79,102578.0,-31143.0,109510.0,-0.27,64.0,H-SC,6.42,151.0,-0.30,0.79,27.51,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.51,-13.84,16.09,0.02,24199.0,-24159.0,150400.0,129.35,48.0,M-SC,14.42,234.0,-1.00,1.08,30.82,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.79,-22.14,93.67,50.79,102578.0,-31143.0,109510.0,-0.27,64.0,H-SC,6.42,151.0,-0.30,0.79,27.51,XR,ATH,IT
84,WIPRO,420.00,0.00,6.32,62.02,72.27,111734.0,10710.0,180158.0,-8.80,63.0,M-LC,2.59,99.0,0.10,1.29,12.89,XR,NTT,IT
41,INFY,1972.00,-0.86,8.44,23.80,34.24,80751.0,26400.0,339288.0,-15.65,59.0,X-LC,4.01,2.0,0.33,2.43,17.42,X40,NTT,IT
73,TCS,4389.97,0.50,-11.43,36.29,20.71,121569.0,-43243.0,334994.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.40,11.93,X40,ATH,IT
1,ABB,7934.00,-0.12,-3.94,51.54,45.57,129522.0,-10315.0,251304.0,-38.27,56.0,H-MC,4.55,120.0,-0.08,1.80,6.93,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.27
1,25,44.87
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.98
LC    24.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.68
X40      23.03
X40N     13.05
XR        9.76
XY25      9.08
AR        9.05
OX40N     7.58
SR        0.99
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.48
H-SC    23.77
X-LC    20.67
M-SC    11.68
X-SC     8.05
H-MC     4.82
L-SC     1.40
M-MC     1.33
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-7.67,42.29
IT,13.01,-15.88,74.15
FINANCE,9.48,-20.60,73.05
MISC,7.27,-28.22,80.67
ELECTRICAL,6.02,-11.85,52.82
PAINTS,5.84,-13.34,30.17
INSURANCE,4.77,-2.00,36.90
PHARMA,4.13,-2.13,34.28
AUTO,2.81,-33.58,79.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3201578.0,21
AR,1309266.0,10
XR,1302775.0,13
X40,1013641.0,14
X40N,866229.0,9
OX40N,733694.0,10
XY25,375302.0,6
SR,283113.0,2
MH,78815.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3666325.0,25
M-SC,1431013.0,15
X-MC,1415122.0,16
X-LC,843576.0,11
X-SC,797181.0,8
H-MC,405761.0,3
L-SC,265025.0,3
M-LC,111734.0,1
H-LC,75756.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1282739.0      6
           AR         895921.0      5
M-SC       XY24       824913.0      6
H-SC       XR         783495.0      7
X-MC       X40        491997.0      7
           XY24       405435.0      3
H-SC       OX40N      342242.0      4
X-MC       X40N       337622.0      4
X-LC       X40        332160.0      5
X-SC       X40N       309621.0      3
M-SC       OX40N      306044.0      5
X-SC       XY24       298076.0      3
H-SC       SR         283113.0      2
X-LC       X40N       218986.0      2
H-MC       AR         210983.0      2
X-LC       XY24       195637.0      2
H-MC       XY24       194778.0      1
X-SC       X40        189484.0      2
X-MC       XY25       180068.0      2
L-SC       XR         179617.0      2
M-SC       XR         173450.0      2
           AR         126606.0      2
M-LC       XR         111734.0      1
X-LC       XY25        96793.0      2
L-SC       OX40N       85408.0      1
H-SC       MH          78815.0      1
H-LC       AR          75756.0      1
M-MC       XY25        60392.0      1
L-MC       XR          54479.0      1
L-LC       XY25        38049.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
